# Contenu:

[+] Données

    + Chargement des open datas + sauvegarde locale

    + Aggregation des tables:
        * Bornes de charge:
            + La table a une colonne, qui est sensée contenir les codes insee des communes, lors de la jointure ça ne colle pas...
            + Solution retenue: spacial join pour affecter le code insee commune des bornes.POINT inscfits dans communes.0OLYGON
            + Contenus agrégés dans des listes (accès aux carateristiques d'un point de charge par son index )
            
        * Pour les autres tables agregation sur code commune INSEE
            + pour les données sur plusieurs années: min, median, max

[+] Communes:

    #   Column                   Non-Null Count   Dtype   
    ---  ------                   --------------   -----   
    0   insee                    314540 non-null  object   code insee commune
    1   nom                      314540 non-null  object   
    2   wikipedia                314504 non-null  object   quelques infos
    3   surf_ha                  314540 non-null  float64 
    4   geometry                 314540 non-null  geometry
    5   PMUN                     314358 non-null  float64  population municipale
    6   PCAP                     314358 non-null  float64  pop domiciliée mais n'habitant pas la commune
    7   PTOT                     314358 non-null  float64  somme des deux précédents
    8   nb_vp                    314535 non-null  float64  vp immat commune
    9   libepci                  314535 non-null  object  
    10  libgeo                   314535 non-null  object  
    11  nb_vp_rechargeables_gaz  314535 non-null  float64 VG 
    12  date_arrete              314535 non-null  object  date du décompte (2021 à 2023)
    13  epci                     314535 non-null  object  
    14  nb_vp_rechargeables_el   314535 non-null  float64 VE 
    15  dep                      314540 non-null  object  
    16  dep_name                 313379 non-null  object  
    17  region_name              313379 non-null  object  
    dtypes: float64(7), geometry(1), object(10)
    memory usage: 45.6+ MB

[+] Bornes

    Int64Index: 55320 entries, 43292 to 36970
    Data columns (total 53 columns):
    #   Column                               Non-Null Count  Dtype              
    ---  ------                               --------------  -----              
    0   nom_amenageur                        55320 non-null  object             
    1   siren_amenageur                      55320 non-null  object             
    2   contact_amenageur                    55320 non-null  object             
    3   nom_operateur                        55320 non-null  object             
    4   contact_operateur                    55320 non-null  object             
    5   telephone_operateur                  55320 non-null  object             
    6   nom_enseigne                         55320 non-null  object             
    7   id_station_itinerance                55320 non-null  object             
    8   id_station_local                     55320 non-null  object             
    9   nom_station                          55320 non-null  object             
    10  implantation_station                 55320 non-null  object             
    11  adresse_station                      55320 non-null  object             
    12  code_insee_commune                   55320 non-null  object             
    13  coordonneesXY                        55320 non-null  object             
    14  nbre_pdc                             55320 non-null  float64            
    15  id_pdc_itinerance                    55320 non-null  object             
    16  id_pdc_local                         55320 non-null  object             
    17  puissance_nominale                   55320 non-null  float64            
    18  prise_type_ef                        55320 non-null  bool               
    19  prise_type_2                         55320 non-null  bool               
    20  prise_type_combo_ccs                 55320 non-null  bool               
    21  prise_type_chademo                   55320 non-null  bool               
    22  prise_type_autre                     55320 non-null  bool               
    23  gratuit                              55320 non-null  bool               
    24  paiement_acte                        55320 non-null  bool               
    25  paiement_cb                          55320 non-null  bool               
    26  paiement_autre                       55320 non-null  bool               
    27  tarification                         55320 non-null  bool               
    28  condition_acces                      55320 non-null  object             
    29  reservation                          55320 non-null  bool               
    30  horaires                             55320 non-null  object             
    31  accessibilite_pmr                    55320 non-null  object             
    32  restriction_gabarit                  55320 non-null  bool               
    33  station_deux_roues                   55320 non-null  bool               
    34  raccordement                         55320 non-null  object             
    35  num_pdl                              55320 non-null  object             
    36  date_mise_en_service                 55320 non-null  object             
    37  observations                         55320 non-null  object             
    38  date_maj                             55320 non-null  object             
    39  cable_t2_attache                     55320 non-null  bool               
    40  last_modified                        55320 non-null  datetime64[ns, UTC]
    41  datagouv_dataset_id                  55320 non-null  object             
    42  datagouv_resource_id                 55320 non-null  object             
    43  datagouv_organization_or_owner       55320 non-null  object             
    44  consolidated_longitude               55320 non-null  float64            
    45  consolidated_latitude                55320 non-null  float64            
    46  consolidated_code_postal             52433 non-null  object             
    47  consolidated_commune                 55320 non-null  object             
    48  consolidated_is_lon_lat_correct      55320 non-null  bool               
    49  consolidated_is_code_insee_verified  55320 non-null  bool               
    50  geometry                             55320 non-null  geometry           
    51  insee                                55320 non-null  object             
    52  dep                                  55320 non-null  object             
    dtypes: bool(16), datetime64[ns, UTC](1), float64(4), geometry(1), object(31)
    memory usage: 16.9+ MB

In [ ]:
# from tqdm.notebook import tqdm
from tqdm import tqdm

import os
os.environ['USE_PYGEOS'] = '0'

# Data management
import pandas as pd
import geopandas as gpd
import numpy as np


# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.features import Choropleth
from folium.plugins import MarkerCluster

# Preprocessing
from sklearn.preprocessing import StandardScaler
import umap


# I/O
import gc
import io, requests
import zipfile, shutil
import joblib

tqdm().pandas()

# data_path = 'C:/Users/demo/Desktop/Lattitude/datas/'
data_path = 'datas'
os.makedirs(data_path, exist_ok=True)

# Datas

## Bornes de recharge

In [ ]:
file_name = 'consolidation-etalab-schema-irve-statique-v-2.2.0-20230327'
ext = '.json'
url = 'https://www.data.gouv.fr/fr/datasets/r/7eee8f09-5d1b-4f48-a304-5e99e8da1e26'

try: 
    print('Loading data from local file...')
    bornes = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
    
except:
    print('Loading data from url...')
    # from url
    bornes = gpd.read_file( url)

    # bornes = gpd.read_file(os.path.join(data_path,file_name + ext))
    print('Saving data to local file...')
    bornes.to_feather(os.path.join(data_path,file_name + '.feather'))

display(bornes.info() )   

In [ ]:
# Create a folium map centered on the first row of datas_
centroid = list(bornes.iloc[0].geometry.centroid.coords[0])
display(centroid, centroid[::-1])


# create a folium map
m = folium.Map(location=[*centroid[::-1]], zoom_start=7)#, crs='EPSG3857')
display(m)
del m

In [ ]:
def convert_strings_bools(val):
    if val.lower() == 'true':
        return True
    elif val.lower() == 'false':
        return False
    else:
        return val
    
# find all columns that contain the string values 'True', 'true', 'False', or 'false',
#  convert those values to boolean, and set the column type to boolean
bool_columns = bornes.applymap(lambda x: isinstance(x, str) and x.lower() in ['true', 'false']).any()
bornes.loc[:, bool_columns] = bornes.loc[:, bool_columns].applymap(convert_strings_bools).astype(bool)
bornes.info()
   

In [ ]:
for col in bornes.select_dtypes(include='object').columns:
    try :
        bornes[col] = bornes[col].astype('float')
    except ValueError:
        continue

bornes.info()

## Types de voitures

In [ ]:
file_name = 'voitures-par-commune-par-energie'
ext = '.geojson'
url = 'http://opendata.agenceore.fr/explore/dataset/voitures-par-commune-par-energie/download?format=geojson&timezone=Europe/Berlin&use_labels_for_header=false'

try: 
    print('Loading data from local file...')
    cars = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
except:
    print('Loading data from url...')
    # from url
    s = requests.get(url).content
    cars = gpd.read_file(io.StringIO(s.decode('utf-8')))

    # avoiding multi dowloads for GES emmision lowering
    print('Saving data to local file...')
    cars.to_feather(os.path.join(data_path,file_name + '.feather'))

print(cars.info())

## Découpage  Administratif


In [ ]:


file_name = 'decoupage_administratif'
ext = '.json'
url ='https://www.data.gouv.fr/fr/datasets/r/fb3580f6-e875-408d-809a-ad22fc418581'
# temp_path = 'temp_unzip'


try: 
    print('Loading data from local file...')
    decoupage = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
except:
    print('Loading data from url...')

    s = requests.get(url).content
    decoupage = pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    print('Saving data to local file...')
    decoupage.to_feather(os.path.join(data_path,file_name + '.feather'))

decoupage.info()




## Departements et régions

In [ ]:
file_name = 'depatements-regions'
ext = '.csv'
url ='https://www.data.gouv.fr/fr/datasets/r/987227fb-dcb2-429e-96af-8979f97c9c84'
# temp_path = 'temp_unzip'


try: 
    print('Loading data from local file...')
    regions = pd.read_feather(os.path.join(data_path,file_name + '.feather'))
except:
    print('Loading data from url...')

    s = requests.get(url).content
    regions = pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    print('Saving data to local file...')
    regions.to_feather(os.path.join(data_path,file_name + '.feather'))

regions.info()

In [ ]:
file_name = 'communes-20220101'
ext = '.shp'
url ='https://www.data.gouv.fr/fr/datasets/r/0e117c06-248f-45e5-8945-0e79d9136165'
temp_path = 'temp_unzip'


try: 
    print('Loading data from local file...')
    communes = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
except:
    print('Loading data from url...')
    # Zip file from url  
    zip_file = requests.get(url).content
    os.makedirs(temp_path, exist_ok=True)
    with zipfile.ZipFile(io.BytesIO(zip_file)) as archive:
        archive.extractall(temp_path)
    communes = gpd.read_file(os.path.join(temp_path,file_name + ext))   
    shutil.rmtree(temp_path) 
    
    print('Saving data to local file...')
    communes.to_feather(os.path.join(data_path,file_name + '.feather'))

communes.info()


# Données démographiques

In [ ]:



file_name = 'Communes'
ext = '.csv'
url ='https://www.insee.fr/fr/statistiques/fichier/4265429/ensemble.zip'
temp_path = 'temp_unzip'


try: 
    print('Loading data from local file...')
    pop_communes = pd.read_feather(os.path.join(data_path,file_name + '.feather'))
except:
    print('Loading data from url...')
    # Zip file from url  
    zip_file = requests.get(url).content
    os.makedirs(temp_path, exist_ok=True)
    with zipfile.ZipFile(io.BytesIO(zip_file)) as archive:
        archive.extractall(temp_path)
    pop_communes = pd.read_csv(os.path.join(temp_path,file_name + ext), sep=';')     
    shutil.rmtree(temp_path) 

    # Rename the columns
    pop_communes.rename(columns={'DEPCOM': 'insee'}, inplace=True)
    
    print('Saving data to local file...')
    pop_communes.to_feather(os.path.join(data_path,file_name + '.feather'))

pop_communes.info()


--------------------

# Join

## Communes and cars

In [ ]:
display(cars.head(2), cars.shape, len(set(cars.codgeo)))

In [ ]:
datas = communes.join(pop_communes.drop('COM',axis=1).set_index('insee'), 
                      on='insee',
                      how='left', 
                      validate='1:1'
                        )\
                .join(cars.rename(columns={'codgeo': 'insee'}).drop('geometry',axis=1).set_index('insee'), 
                      on='insee', 
                      how='left', 
                      # rsuffix='_cars',
                      validate='1:m'
                      )\
            #     .join(bornes.rename(columns={'consolidated_code_postal': 'insee'}).set_index('insee'), 
            #           on='insee', 
            #           how='left', 
            #           rsuffix='_bornes',
            #           validate='m:m')

del pop_communes            
datas.info()

## regions

Création dep en prennant les deux premiers str de insee

In [ ]:
datas['dep'] = datas['insee'].apply(lambda s : s[0:2])


display(regions.head(2), regions.shape, len(set(datas.dep)))

set(regions.num_dep) - set(datas.dep)

In [ ]:
datas = datas.join(regions.rename(columns={'num_dep': 'dep'}).set_index('dep'), on='dep', how='left', validate='m:1')

# del regions
datas.info()

# Join bornes de charge

In [ ]:
bornes.sort_values(by='date_mise_en_service', inplace=True)
display(bornes.head(2), bornes.shape, len(set(bornes.consolidated_code_postal)))

55 320 bornes réparties sur seulement 3 119 communes

en reprennant les codes insee et polygons de datas,  creation de insee dans bornes

In [ ]:
# Convert the GeoDataFrames to a dask DataFrame
com_ = communes[['insee','geometry']]
bor_ = bornes[['consolidated_code_postal','geometry']]


# display(com_.shape[0], bor_.shape[0])

bornes['insee'] = gpd.sjoin(com_, bor_,  how='right', predicate='contains',lsuffix='_com', rsuffix='_bor')['insee'].astype(str)


In [ ]:
# Add dep, dep_name, region_name

bornes['dep'] = bornes['insee'].apply(lambda s : s[0:2])

bornes = bornes.join(regions.rename(columns={'num_dep': 'dep'}).set_index('dep'), on='dep', how='left', validate='m:1')



# del regions, communes, com_, bor_
bornes.info()

In [ ]:
# check Nan
bornes[bornes.insee.isna()].consolidated_is_lon_lat_correct.value_counts()

------------------------------------------------

# Save to local

In [ ]:
# save datas
file_name = 'dataset_communes'
print('Saving datas to local file...')
# datas.to_file(os.path.join(data_path,file_name + '.geojson'), driver='GeoJSON') # > 5 Go
# datas.to_parquet(os.path.join(data_path,file_name + '.parquet')) # 1.5 Go
datas.to_feather(os.path.join(data_path,file_name + '.feather'), compression='zstd') # 420 Mo

In [ ]:
# save bornes
file_name = 'dataset_charge_points'
print('Saving bornes to local file...')
bornes.to_feather(os.path.join(data_path,file_name + '.feather'), compression='zstd')